# 3M dataset - Data Preprocessing

 cleansing 3M flight records for delay prediction modeling.

## Contents
1. [Data Loading](#loading)
2. [Handling cancelled flihgts](#cancelled)
3. [Save Processed Dataset](#save)
4. [Handling Data Types](#data_types)
5. [Handling Missing Values](#missing_values)


## 1. Data Loading <a id='loading'></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(PROJECT_ROOT)
from src.data import loader, processor
from src.visualization import exploratory_viz

In [2]:
raw_file_path = os.path.join(PROJECT_ROOT, 'data', 'raw', 'flights_sample_3m.csv')
processed_dir = os.path.join(PROJECT_ROOT, 'data', 'processed')
os.makedirs(processed_dir, exist_ok=True)

print(f"Input data file: {raw_file_path}")
print(f"Processed data directory: {processed_dir}")

Input data file: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\raw\flights_sample_3m.csv
Processed data directory: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\processed


In [3]:
# Function to process data in chunks to handle 3M rows efficiently
def process_data_in_chunks(file_path, chunksize=100000):
    """
    Generator function to process the data in chunks
    Args:
        file_path: Path to the CSV file
        chunksize: Number of rows to process at a time
    Yields:
        Processed DataFrame chunks
    """
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        yield chunk

first_chunk = next(process_data_in_chunks(raw_file_path, chunksize=5))
first_chunk

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


## 2. Handling Cancelled Flights <a id='cancelled'></a>

As we're focusing on predicting delays for flights that actually operated, we'll separate cancelled flights from our analysis dataset. Cancelled flights represent a different phenomenon and would confuse our delay prediction models.

In [4]:
# First, let's analyze how many flights were cancelled
def analyze_cancellations(file_path, chunksize=100000):
    cancelled_count = 0
    total_count = 0
    cancellation_codes = {}
    
    # Process in chunks to handle the large dataset
    for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize)):
        total_count += len(chunk)
        
        # Count cancelled flights
        cancelled_in_chunk = chunk[chunk['CANCELLED'] == 1.0]
        cancelled_count += len(cancelled_in_chunk)
        
        # Track cancellation reasons
        code_counts = cancelled_in_chunk['CANCELLATION_CODE'].value_counts()
        for code, count in code_counts.items():
            if code in cancellation_codes:
                cancellation_codes[code] += count
            else:
                cancellation_codes[code] = count
                
        # Print progress
        if (i+1) % 10 == 0:
            print(f"Processed {(i+1)*chunksize:,} rows...")
    
    # Calculate percentage
    cancelled_pct = (cancelled_count / total_count) * 100 if total_count > 0 else 0
    
    # Create a summary dictionary
    summary = {
        'total_flights': total_count,
        'cancelled_flights': cancelled_count,
        'cancelled_percentage': cancelled_pct,
        'cancellation_codes': cancellation_codes
    }
    
    return summary

# Run the analysis
cancellation_summary = analyze_cancellations(raw_file_path)

# Display the results
print(f"Total Flights: {cancellation_summary['total_flights']:,}")
print(f"Cancelled Flights: {cancellation_summary['cancelled_flights']:,} ({cancellation_summary['cancelled_percentage']:.2f}%)")
print("\nCancellation Codes:")
for code, count in cancellation_summary['cancellation_codes'].items():
    print(f"  {code}: {count:,} flights")
    
# Explanation of cancellation codes
code_meanings = {
    'A': 'Carrier',
    'B': 'Weather',
    'C': 'National Air System',
    'D': 'Security'
}

print("\nCancellation Code Meanings:")
for code, meaning in code_meanings.items():
    print(f"  {code}: {meaning}")

Processed 1,000,000 rows...
Processed 2,000,000 rows...
Processed 3,000,000 rows...
Total Flights: 3,000,000
Cancelled Flights: 79,140 (2.64%)

Cancellation Codes:
  B: 28,772 flights
  D: 24,417 flights
  A: 19,476 flights
  C: 6,475 flights

Cancellation Code Meanings:
  A: Carrier
  B: Weather
  C: National Air System
  D: Security


In [20]:
# Function to filter out cancelled flights and save operational flights
def filter_operational_flights(input_path, output_path, chunksize=100000):
    """
    Filter the dataset to include only operational flights (not cancelled)
    and save it to a new CSV file.
    
    Args:
        input_path: Path to the input CSV file
        output_path: Path to save the filtered CSV file
        chunksize: Number of rows to process at a time
    """
    # Track statistics
    total_rows = 0
    operational_rows = 0
    
    # Process the first chunk to get the header
    first_chunk = True
    
    # Process in chunks
    for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
        # Update counts
        total_rows += len(chunk)
        
        # Filter out cancelled flights
        operational_chunk = chunk[chunk['CANCELLED'] != 1.0]
        operational_rows += len(operational_chunk)
        
        # Save the chunk (append mode after first chunk)
        if first_chunk:
            operational_chunk.to_csv(output_path, index=False)
            first_chunk = False
        else:
            operational_chunk.to_csv(output_path, mode='a', header=False, index=False)
        
        # Print progress
        if (i+1) % 5 == 0:
            print(f"Processed {(i+1)*chunksize:,} rows...")
    
    # Return statistics
    return {
        'total_rows': total_rows,
        'operational_rows': operational_rows,
        'filtered_rows': total_rows - operational_rows,
        'filtered_percentage': ((total_rows - operational_rows) / total_rows) * 100 if total_rows > 0 else 0
    }

operational_flights_path = os.path.join(processed_dir, 'operational_flights.csv')

# Run the filtering process
print("Filtering out cancelled flights...")
filter_stats = filter_operational_flights(raw_file_path, operational_flights_path)

# Display the results
print("\nFiltering complete!")
print(f"Total rows processed: {filter_stats['total_rows']:,}")
print(f"Operational flights: {filter_stats['operational_rows']:,}")
print(f"Cancelled flights removed: {filter_stats['filtered_rows']:,} ({filter_stats['filtered_percentage']:.2f}%)")
print(f"\nOperational flights saved to: {operational_flights_path}")

Filtering out cancelled flights...
Processed 500,000 rows...
Processed 1,000,000 rows...
Processed 1,500,000 rows...
Processed 2,000,000 rows...
Processed 2,500,000 rows...
Processed 3,000,000 rows...

Filtering complete!
Total rows processed: 3,000,000
Operational flights: 2,920,860
Cancelled flights removed: 79,140 (2.64%)

Operational flights saved to: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\processed\operational_flights.csv


## 4. Handling Data Types<a id='data_types'></a>

changing some features data type to datetime datatype (takes time)

In [ ]:
import pandas as pd
import os
import tempfile
import shutil

def convert_flight_times_to_datetime(input_path, chunksize=100_000):
    """
    Efficiently convert FL_DATE and hhmm time columns to datetime for large CSVs in chunks.
    Saves results to a temp file and replaces the original once complete.
    """
    time_columns = ['DEP_TIME', 'ARR_TIME', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'WHEELS_OFF', 'WHEELS_ON']

    # Create temporary file
    dir_name = os.path.dirname(input_path)
    temp_fd, temp_path = tempfile.mkstemp(dir=dir_name, suffix=".csv")
    os.close(temp_fd)

    total_rows = 0
    first_chunk = True

    for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
        total_rows += len(chunk)

        # Convert FL_DATE to datetime
        chunk['FL_DATE'] = pd.to_datetime(chunk['FL_DATE'], errors='coerce')

        for col in time_columns:
            if col in chunk.columns:
                # Work on a copy of the time column as strings
                time_raw = pd.to_numeric(chunk[col], errors='coerce').dropna().astype(int).astype(str).str.zfill(4)

                # Prepare full datetime strings
                valid_mask = chunk['FL_DATE'].notna() & chunk[col].notna()
                combined_str = (
                    chunk.loc[valid_mask, 'FL_DATE'].dt.strftime('%Y-%m-%d') + ' ' +
                    time_raw.loc[valid_mask].str[:2] + ':' + time_raw.loc[valid_mask].str[2:]
                )

                # Initialize as NaT
                chunk[col] = pd.NaT

                # Assign parsed datetime values
                chunk.loc[valid_mask, col] = pd.to_datetime(combined_str, errors='coerce')

        # Write chunk to temp file
        if first_chunk:
            chunk.to_csv(temp_path, index=False)
            first_chunk = False
        else:
            chunk.to_csv(temp_path, mode='a', header=False, index=False)

        # Print progress every 5 chunks
        if (i + 1) % 5 == 0:
            print(f"✅ Processed {(i + 1) * chunksize:,} rows so far...")

    # Replace original file
    shutil.move(temp_path, input_path)
    print(f"\n Cleaned file saved to: {input_path}")
    print(f"📊 Total rows processed: {total_rows:,}")




convert_flight_times_to_datetime(operational_flights_path)


✅ Processed 500,000 rows so far...
✅ Processed 1,000,000 rows so far...
✅ Processed 1,500,000 rows so far...
✅ Processed 2,000,000 rows so far...
✅ Processed 2,500,000 rows so far...
✅ Processed 3,000,000 rows so far...

🏁 Done! Cleaned file saved to: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\processed\operational_flights.csv
📊 Total rows processed: 2,920,860


## 5. Handling Missing Values <a id='missing_values'></a>

In [22]:
import pandas as pd

def check_missing_values_in_chunks(file_path, chunk_size=100000):
    """
    Processes a large CSV file in chunks to compute missing values per column.

    Args:
        file_path: Path to the input CSV file.
        chunk_size: Number of rows to process per chunk.

    Returns:
        DataFrame summarizing missing value counts and percentages per column.
    """
    total_rows = 0
    missing_counts = {}
    data_types = None

    for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
        total_rows += len(chunk)

        if i == 0:
            data_types = chunk.dtypes

        for column in chunk.columns:
            if column not in missing_counts:
                missing_counts[column] = 0
            missing_counts[column] += chunk[column].isna().sum()

        if (i + 1) % 5 == 0:
            print(f"✅ Processed {(i + 1) * chunk_size:,} rows so far...")

    missing_summary = pd.DataFrame({
        'Column': list(missing_counts.keys()),
        'Data_Type': [data_types[col] for col in missing_counts.keys()],
        'Missing_Count': list(missing_counts.values())
    })

    missing_summary['Missing_Percentage'] = (missing_summary['Missing_Count'] / total_rows) * 100
    missing_summary = missing_summary.sort_values(by='Missing_Percentage', ascending=False).reset_index(drop=True)

    return missing_summary




operational_flights_path = os.path.join(processed_dir, 'operational_flights.csv')
missing_summary = check_missing_values_in_chunks(operational_flights_path, chunk_size=100000) 
missing_summary

✅ Processed 500,000 rows so far...
✅ Processed 1,000,000 rows so far...
✅ Processed 1,500,000 rows so far...
✅ Processed 2,000,000 rows so far...
✅ Processed 2,500,000 rows so far...
✅ Processed 3,000,000 rows so far...


,Column,Data_Type,Missing_Count,Missing_Percentage
0,CANCELLATION_CODE,float64,2920860,100.000000
1,DELAY_DUE_LATE_AIRCRAFT,float64,2386997,81.722404
2,DELAY_DUE_CARRIER,float64,2386997,81.722404
3,DELAY_DUE_SECURITY,float64,2386997,81.722404
4,DELAY_DUE_NAS,float64,2386997,81.722404
5,DELAY_DUE_WEATHER,float64,2386997,81.722404
6,ARR_DELAY,float64,7058,0.241641
7,ELAPSED_TIME,float64,7058,0.241641
8,AIR_TIME,float64,7058,0.241641
9,ARR_TIME,object,2218,0.075937


In [23]:
import shutil
import pandas as pd
import os

def handle_missing_values(file_path, chunk_size=100000):
    abs_input = os.path.abspath(file_path)
    output_path = abs_input.replace(".csv", "_cleaned.csv")
    print(f"\n📝 Input file: {abs_input}")
    print(f"📁 Temp output file: {output_path}")

    if os.path.exists(output_path):
        os.remove(output_path)

    total_rows_processed = 0
    chunk_count = 0

    stats = {
        'delay': {},
        'numeric': {},
        'categorical': {},
        'datetime': {}
    }

    datetime_medians = {}

    for i, chunk in enumerate(pd.read_csv(abs_input, chunksize=chunk_size, parse_dates=True)):
        chunk_count += 1
        rows = len(chunk)
        total_rows_processed += rows
        if (i + 1) % 5 == 0:
            print(f"✅ Processed {(i + 1) * chunk_size:,} rows so far...")

        df_clean = chunk.copy()

        if 'CANCELLATION_CODE' in df_clean.columns:
            df_clean = df_clean.drop(columns=['CANCELLATION_CODE'])

        delay_reason_cols = [col for col in df_clean.columns if 'DELAY_DUE_' in col]
        for col in delay_reason_cols:
            missing_count = df_clean[col].isna().sum()
            if missing_count > 0:
                df_clean[col] = df_clean[col].fillna(0)
                stats['delay'][col] = stats['delay'].get(col, 0) + missing_count

        numeric_cols = df_clean.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            if col not in delay_reason_cols:
                missing_count = df_clean[col].isna().sum()
                if missing_count > 0:
                    median_value = df_clean[col].median()
                    df_clean[col] = df_clean[col].fillna(median_value)
                    if col not in stats['numeric']:
                        stats['numeric'][col] = {'count': 0, 'median': median_value}
                    stats['numeric'][col]['count'] += missing_count

        categorical_cols = df_clean.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            missing_count = df_clean[col].isna().sum()
            if missing_count > 0:
                mode_value = df_clean[col].mode()[0]
                df_clean[col] = df_clean[col].fillna(mode_value)
                if col not in stats['categorical']:
                    stats['categorical'][col] = {'count': 0, 'mode': mode_value}
                stats['categorical'][col]['count'] += missing_count

        datetime_cols = df_clean.select_dtypes(include=['datetime64[ns]']).columns
        for col in datetime_cols:
            missing_count = df_clean[col].isna().sum()
            if missing_count > 0:
                if col not in datetime_medians:
                    datetime_medians[col] = df_clean[col].dropna().median()
                df_clean[col] = df_clean[col].fillna(datetime_medians[col])
                stats['datetime'][col] = stats['datetime'].get(col, 0) + missing_count

        df_clean.to_csv(output_path, mode='a', index=False, header=(i == 0))

    print("\n===== FINAL CLEANING SUMMARY =====")
    print(f"Total chunks processed: {chunk_count}")
    print(f"Total rows processed: {total_rows_processed}")

    if stats['delay']:
        print("\nDelay columns filled with 0:")
        for col, count in stats['delay'].items():
            print(f"  {col}: {count} values filled")

    if stats['numeric']:
        print("\nNumeric columns filled with median:")
        for col, info in stats['numeric'].items():
            print(f"  {col}: {info['count']} values filled, median used = {info['median']:.4f}")

    if stats['categorical']:
        print("\nCategorical columns filled with mode:")
        for col, info in stats['categorical'].items():
            print(f"  {col}: {info['count']} values filled, mode used = {info['mode']}")

    if stats['datetime']:
        print("\nDatetime columns filled with median:")
        for col, count in stats['datetime'].items():
            print(f"  {col}: {count} values filled, median used = {datetime_medians[col]}")

    shutil.move(output_path, abs_input)
    print(f"\n✅ Original file replaced with cleaned data: {abs_input}")

    return {
        "total_rows": total_rows_processed,
        "total_chunks": chunk_count,
        "summary": stats
    }


# Run the function
result = handle_missing_values(operational_flights_path, chunk_size=100000)



📝 Input file: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\processed\operational_flights.csv
📁 Temp output file: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\processed\operational_flights_cleaned.csv
✅ Processed 500,000 rows so far...
✅ Processed 1,000,000 rows so far...
✅ Processed 1,500,000 rows so far...
✅ Processed 2,000,000 rows so far...
✅ Processed 2,500,000 rows so far...
✅ Processed 3,000,000 rows so far...

===== FINAL CLEANING SUMMARY =====
Total chunks processed: 30
Total rows processed: 2920860

Delay columns filled with 0:
  DELAY_DUE_CARRIER: 2386997 values filled
  DELAY_DUE_WEATHER: 2386997 values filled
  DELAY_DUE_NAS: 2386997 values filled
  DELAY_DUE_SECURITY: 2386997 values filled
  DELAY_DUE_LATE_AIRCRAFT: 2386997 values filled

Numeric columns filled with median:
  TAXI_IN: 804 values filled, median used = 6.0000
  ARR_DELAY: 7058 values filled, median used = -7.0000
  ELAPSED_TIME: 7058 values filled, median used = 120.0000
  AIR_TIME: 